# Bag of Popcorns을 RNN으로 해보자! (feat. Pytorch)

  이번 스터디에서는 RNN을 활용하여 Movie Review에 대한 감정분석을 할 예정입니다. 모델은 Pytorch로 구현하였고, 학습 데이터 구성은 Torchtext를 활용하였습니다. 제가 이 스터디를 준비하는데 활용한 레퍼런스는 아래에 작성해두었습니다.

## 목차

1. 데이터 셋업
2. 모델 구현
3. 학습 준비
4. 모델 저장
5. 모델 로드
6. 테스트

## Reference

- Torchtext 예제: http://anie.me/On-Torchtext/
- Pytorch Tutorial 예제: https://github.com/yunjey/pytorch-tutorial

In [ ]:
from torchtext import data
import nltk
from bs4 import BeautifulSoup
from tqdm import tqdm

## 데이터 셋업

데이터 셋업은 'torchtext' 라이브러리를 활용하여 모델에 넣을 Input Data를 준비합니다. 
아마도 torchtext를 활용하는데 생소하신 분들이 많을 겁니다.
그래서, 여기서는 각 활용하는 부분에 대해서 라인별로 각각 설명하고자 합니다.

### Tokenize 정의

Torchtext에서 기본으로 정의된 tokenize 방식은 `str.split()` 입니다. 즉, space를 기준으로 토크나이즈 하는 방식입니다. 경우에 따라서는 개발자가 정의한 토크나이즈 방식이 있습니다. 그런 경우에는 tokenize 함수를 정의하서 활용하도록 합니다. 아래의 경우에는 BeautifulSour을 가지고 Text에 html 양식 내용을 제거하고 nltk word tokenizer를 활용하여 토크나이즈를 하였습니다. 

In [ ]:
def tokenize(text):
    pure_text = BeautifulSoup(text,"html5lib").get_text()
    tokenized = nltk.word_tokenize(pure_text)
    return tokenized

### data.Field 정의

`data.Field`은 데이터 각 인스턴트에 대해서 모델에 어떠한 양식으로 입력할 것인지에 대해서 정의 하는 모듈입니다.
`TEXT` 변수의 경우는 Movie Review 데이터 인스턴스에 대한 Input 양식을 정의합니다. `LABEL`의 경우는 Sentiment에 대한 레이블링입니다.

각 경우에 대해서 설정을 아래 Field내 파라메터와 같이 세팅을 할 수 있고, 자세한 설정에 관한 부분은 아래 링크에서 확인 할 수 있습니다. 

- data.Field에 관한 메뉴얼:
http://torchtext.readthedocs.io/en/latest/data.html#fields

In [ ]:
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, 
                  batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)

### data.Dataset 정의

data.Dataset은 데이터셋 파일을 읽어들이는 모듈입니다. 데이터셋을 읽어들일 경로를 지정하고 해당하는 데이터 column에 대한 정의를 tag 이름과 위에서 정의한 data.Field 객체로 튜플화 해서 정해주면 됩니다. 그 외에 다른 옵션들이 있는데 format의 경우는 가급적 명시를 하고 파일을 읽어들이는게 좋습니다. 

지금 아래의 경우는 Dataset을 상속받은 TabularDataset 객체로 선언이 되어있는데 주로 Classification을 다루는데 유용한 모듈입니다. 만일 train과 test 데이터셋에 대한 load를 동시에 하고자 한다면 `splits()`함수를 활용하여 동시에 받을 수 있습니다. 

더 자세한 설정에 관한 부분은 아래 링크에서 확인 할 수 있습니다.

- data.Dataset에 관한 메뉴얼: http://torchtext.readthedocs.io/en/latest/data.html#dataset

In [ ]:
train, dev = data.TabularDataset.splits(
              path='./src', train='labeledTrainData.csv', validation='labeledDevData.csv', format='csv',
              skip_header=True, fields=[('Label', LABEL), ('Text', TEXT)])

### Vocabulary 세팅

데이터셋을 불러온 다음에는 주어진 데이터셋 안에서 Vocabulary를 생성할 수 있습니다. 방법은 위에서 생성한 `TEXT`라는 `Field` 객체에 `train` 데이터셋을 주입하여 Vocabulary를 생성하도록 합니다. 아래 경우에는 `vectors` 파라메터를 추가로 두고 `"glove.6B.100d"`를 입력하였는데 이 경우에는 pretrain된 word embedding vector (GLOVE)를 활용하도록 합니다. Pretrain된 벡터 데이터를 캐시에 저장해두도록 하는데는 약간의 시간소요가 있습니다.

In [ ]:
TEXT.build_vocab(train, vectors="glove.6B.100d")

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

## 모델 구현

Sentiment Analysis를 위해 여기서는 RNN을 활용하여 Sentiment Classify를 합니다. 사용하고자 하는 딥러닝 프레임워크는 pytorch로 모델링을 한 객체에서 수행할 수 있도록 합니다. 여기서는 `SentimentAnalyzer`라는 class를 선언하여 네트워크를 구성하였다. 네트워크는 Word Embedding을 RNN에 입력하여 가장 마지막 time step의 hidden vector를 통해 `positive`인지 `negative`를 결정 할 수 있도록 합니다. 여기서 hidden vector의 차원을 `positive`, `negative`로 2차원으로 변환하기 위해 Linear Transform을 하도록 하였습니다.

자세한 구현에 대한 내용은 아래 코드 주석에 달아 두었습니다.

In [ ]:
class SentimentAnalyzer(nn.Module):
    def __init__(self,
                 vocab_size,
                 embedding_dim,
                 hidden_dim,
                 num_layers,
                 embedding_matrix=None):
        super(SentimentAnalyzer, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.use_cuda = False
        
        # Word Embedding Layer: GloVe 모델을 활용하여 단어에 대한 Embedding vector를 냄.
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if embedding_matrix is not None:
            self.embedding.weight.data.copy_(embedding_matrix)
            self.embedding.weight.requires_grad = False
        
        # RNN Modle(GRU Version): 매 time step에 대한 hidden vector를 내고자 함.
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers,
                         batch_first=True, dropout=0.2, bidirectional=True)
        # Linear Layer: 마지막 hidden vector에 대해서 positive, negative에 대한 label을 위해
        #               차원 축소를 함.
        self.linear = nn.Linear(hidden_dim*2, 2)

    def forward(self, inputs):
        batch_size = inputs.size(0)

        embed_vectors = self.embedding(inputs)

        hidden_state = self.init_hidden(batch_size)
        hidden_vectors, _ = self.rnn(embed_vectors, hidden_state)
        logits = self.linear(hidden_vectors[:, -1])
    
        # 예측을 label에 대한 확률을 확인하기 위해 softmax를 취함.
        predicts = F.softmax(logits) 
        # 학습시 Negative Log Likelihood를 적용하기 위해 log softmax를 취함.
        log_logits = F.log_softmax(logits, dim=-1)

        return predicts, log_logits
    
    # initial hidden vector에 대해 cuda 설정이 필요하기 때문에 이와 같은 함수를 둠.
    def set_cuda(self):
        self.use_cuda = True

    def init_hidden(self, batch_size):
        hidden = Variable(torch.zeros(self.num_layers*2, batch_size, self.hidden_dim))
        if self.use_cuda:
            return hidden.cuda()
        return hidden

### 네트워크 모델 생성

위에서 작성한 `SentimentAnalyzer`를 실제로 객체로 생성하기 위해 다음과 같기 구현합니다. 각 생성자에 대한 파라미터에 대한 설명은 아래 코드에 주석을 확인해주시면 됩니다.

In [ ]:
vocab = TEXT.vocab

In [ ]:
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 150
num_layers = 2
embedding_matrix = vocab.vectors

In [ ]:
analyzer = SentimentAnalyzer(vocab_size,
                         embedding_dim,
                         hidden_dim,
                         num_layers,
                         embedding_matrix)

## 학습 준비

### data.Iterator

data.Iterator은 데이터 학습에 iteration을 해주도록 하는 모듈입니다. Iteration에서 기본적으로 설정해야 하는 파라메터는 batch_size와 device, shuffle 정도가 있습니다. batch_size는 한 번 학습을 할 때 얼마만큼의 인스턴스를 연산시킬 것인지에 대한 설정을 합니다. device는 cpu나 gpu활용 세팅을 하는데 필요한 세팅입니다. 기본적으로 cpu경우는 `device=-1`로 세팅을 해야 가능합니다. 아래의 경우는 BucketIterator를 활용하게 되는데 일반적으로 Sequencial Data는 한 Batch를 구성할 때 Sequence Length의 분포에 따라서 학습 속도가 달라질 수 있습니다. 학습 속도를 향상시키기 위해 Bucketing이라는 방법을 활용하는데 이러한 기능이 기본적으로 내장되어있는 Iterator입니다. 

보시다시피 아래 코드에 보면 2가지 케이스로 구분을 해 두었습니다. 이는 gpu 설정이 필요한 경우인지 아닌지를 `torch.cuda.is_available()`를 가지고 판단하여 세팅하도록 하였습니다.

In [ ]:
if torch.cuda.is_available():
    analyzer.set_cuda()
    analyzer.cuda()
    train_iter, dev_iter = data.BucketIterator.splits((train, dev), batch_size=16,
                                 sort_key=lambda x: len(x.Text),
                                 device=0, sort_within_batch=True,
                                 repeat=False)
else:
    train_iter, dev_iter = data.BucketIterator.splits((train, dev), batch_size=16,
                                 sort_key=lambda x: len(x.Text),
                                 device=-1, sort_within_batch=True,
                                 repeat=False)

### Loss & Optimizer

Loss의 경우는 Negative Log Likelihood를 활용하였고, Optimizer는 Adam을 활용하였습니다. Adam Optimizer 파라메터 안에 filter 함수는 모델 내 trainable한 parameter에 대해서 optimize를 하기 위한 방법입니다.

In [ ]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, analyzer.parameters()))

### Avearge Meter class & Accuracy function

Average Meter는 매 Iteration에 나오는 loss나 accuracy를 평균내주는 모듈입니다. 그리고 Accuracy function은 해당 minibatch에 대한 accuracy를 내는 함수힙니다. 이 코드들은 꽤 편하게 모듈을 짜두어서 여러모로 쓰기 편합니다. 코드 참조는 다음 페이지에서 했습니다.

- 코드 Reference: https://github.com/pytorch/examples/blob/master/imagenet/main.py

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    target = target.long()

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    target = target.view(1, -1)
    correct = pred.eq(target)

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## 학습 

In [ ]:
num_epochs = 5
# 매 학습시에 model.train()을 선언해주어야 합니다.
analyzer.train()
for epoch in range(num_epochs):
    acc = AverageMeter()
    _loss = AverageMeter()
    for i, data in enumerate(tqdm(train_iter)):
        # 학습 input data는 data.label이름 형태로 확인할 수 있습니다.
        text, labels = data.Text, data.Label 

        # 학습 전 gradient에 대한 초기화를 해줍니다.
        optimizer.zero_grad()
        predicts, log_logits = analyzer(text)
        # Loss값을 산출합니다.
        loss = criterion(log_logits, labels)
        # Loss값에 대해 Backprop 연산을 합니다.
        loss.backward()
        # Optimizer에 따라 Parameter에 대해 Backprop 연산을 적용합니다.
        optimizer.step()

        temp_acc = accuracy(predicts, labels)
        acc.update(temp_acc[0], text.size(0))
        _loss.update(loss.data[0], text.size(0))
        if (i+1) % 3 == 0:
            print ('Epoch: %d Step: %d Loss: %.4f Accuracy: %.2f'%(epoch, i+1, _loss.avg, acc.avg))

            val_acc = AverageMeter()
            # 매 예측시에 model.eval()을 선언해주어야 합니다.
            analyzer.eval()
            for i, data in enumerate(tqdm(dev_iter)):
                text, labels = data.Text, data.Label
                predicts, _ = analyzer(text)
                temp_acc = accuracy(predicts, labels)
                val_acc.update(temp_acc[0], text.size(0))
            print ('Validation Accuracy: %.2f'%(acc.avg))
            analyzer.train()

## 모델 저장

모델은 다음에 대한 내용을 저장하였습니다. 저장 방법은 torch.save()함수를 사용하였고 저장형태는 json형태로 저장됩니다.

- epoch 횟수
- 모델 파라메터
- 정확도
- 모델 optimizer
- 모델에 대한 vocabulary set
- 모델 하이퍼파라메터

In [ ]:
file_path = 'save_model_%d.pth.tar' % (epoch + 1)
torch.save({'epoch': epoch + 1,
            'state_dict': analyzer.state_dict(),
            'accuracy': acc.avg,
            'optimizer' : optimizer.state_dict(),
            'vocab': vocab.stoi,
            'hyper_params': hyper_params}, file_path)

## 성능 측정

To be continued...

In [ ]:
load_path = './model/save_model_67.pth.tar'
model_json = torch.load(load_path)
vocab = model_json['vocab']

In [ ]:
import pandas as pd
from torch.autograd import Variable
import torch

In [ ]:
test = pd.read_csv('./src/testData.csv')

In [ ]:
id_list = list(test['id'])
test_list = list(test['review'])

In [ ]:
pred = list()

In [ ]:
for item in test_list:
    tokenized = tokenize(item)
    input = Variable(torch.LongTensor([vocab[word] if word in vocab else vocab['unk'] for word in tokenized]))
    output = analyzer(input.unsqueeze(0))
    pred.append((output.topk(1)[1].data.numpy()[0][0])

In [ ]:
answer = {'id': id_list, 'sentiment': pred}
answer_df = pd.DataFrame(answer)

In [ ]:
answer_path = 'answer.csv'
answer_df.to_csv(answer_path)